In [2]:
from rdflib import Namespace, Graph, Literal, URIRef
import re
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes
from buildingmotif.model_builder import TemplateBuilderContext
import os 
import sys 
# %%
# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://")

# create the model w/ a namespace
WBS = Namespace("urn:ex/")
S223 = Namespace("http://data.ashrae.org/standard223#")
wbs = Model.create(WBS)
bind_prefixes(wbs.graph)
wbs.graph.bind("wbs", WBS)
wbs.graph.bind("s223", S223)
wbs.graph.bind("nawi", Namespace("urn:nawi-water-ontology#"))
things = []

AttributeError: module 'pkgutil' has no attribute 'ImpImporter'

In [ ]:
sys.path.append('..')
from utils import * # ModelBuilder and vars

In [ ]:
# %%
templates = Library.load(directory='../templates')
s223 = Library.load(ontology_graph="../../s223/collections/MODEL_SP223_all-v1.0.ttl")
templates.get_templates()

/Users/lazlopaul/Desktop/NAWI/water-ontology/.venv/lib/python3.10/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))
2025-01-23 13:58:23,942 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/vocab/enumeration (No row was found when one was required). Check this is loaded into BuildingMOTIF
2025-01-23 13:58:23,943 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/model/equipment (No row was found when one was required). Check this is loaded into BuildingMOTIF
2025-01-23 13:58:23,943 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/extensions/settings (No row was found when one was required). Check this is loaded into BuildingMOTIF
2025-01-23 13:5

[Template(_id=1, _name='brine', body=<Graph identifier=1f258bc6-55f2-4922-ab59-d54252f4c047 (<class 'rdflib.graph.Graph'>)>, optional_args=['constituent-salt', 'constituent-water'], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10f30ebc0>),
 Template(_id=2, _name='constituent-salt', body=<Graph identifier=2be4890f-d134-4287-84f2-51aa3e6d1093 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10f30ebc0>),
 Template(_id=3, _name='constituent-water', body=<Graph identifier=3b278f46-1012-48b2-8873-086281bb587d (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10f30ebc0>),
 Template(_id=4, _name='ph', body=<Graph identifier=f7bb123c-e8b6-4c59-9986-dccf850365ed (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x10f30ebc0>),
 Template(_id=5, 

In [ ]:
builder = ModelBuilder(ns = WBS, bm_instance= wbs, templates = templates)

In [ ]:
dosing_pump = builder.create_equipment('supply-tank', 'tank', role = S223['Role-Supply'])
# may need a different way of doing differential pressure, but currently don't have enough info to say where differentials are measured
# unclear if speed is a command or just a measured variable
dosing_pump_props = builder.add_properties(dosing_pump['out'], ['flow-rate','speed-command'], ['92','66'])
# TODO: double check if Bypass is a role 
uf_bypass_valve_1 = builder.create_equipment('uf-bypass', 'valve', role = S223['Role-Bypass'])
uf_bypass_valve_1_to_feed = builder.add_connection_point(uf_bypass_valve_1, 'outlet-cp')
builder.connect_equipment(dosing_pump, uf_bypass_valve_1)

break_tank = builder.create_equipment('uf-feed-tank', 'tank', role = S223['Role-Supply'])
builder.connect_equipment(uf_bypass_valve_1, break_tank, source_cp=uf_bypass_valve_1_to_feed)
# TODO: Add units (e.g. gal, C) to the properties that need it. Should maybe have different templates for different units
break_tank_props = builder.add_properties(break_tank['name'], ['volume', 'temperature'], ['17','22'])
# what does OF mean? overfull, over feed? 
# TODO: Should there be a role for OF?
# TODO: Should we use a boundary connection point for the of pump, or create a 'waste' tank or domain space
of_pump = builder.create_equipment('of_pump','pump')
break_tank_to_waste = builder.add_connection_point(break_tank, 'outlet-cp')
builder.connect_equipment(break_tank, of_pump, source_cp=break_tank_to_waste)
# adding a waste tank to represent where waste goes
waste_tank = builder.create_equipment('waste','tank')
# For 223P, waste should probably be a connection that is many inlet connection points to the waste receptical 
waste =builder.connect_equipment(of_pump, waste_tank)
builder.graph.add((waste['name'], RDFS['comment'], Literal('Waste Connections')))

# not sure we should still be using trhe role supply or not
feed_valve = builder.create_equipment('feed-valve','valve', role = S223['Role-Supply'])
feed_valve_drain_cp = builder.add_connection_point(feed_valve, 'outlet-cp')
builder.connect_equipment(break_tank, feed_valve)

feed_pump = builder.create_equipment('feed-pump','pump')
builder.connect_equipment(feed_valve, feed_pump)
feed_pump_props = builder.add_properties(feed_pump['name'], ['speed-command', 'electric-current'], ['59','21'])
# TODO: make a flow rate that is in units of GPM (gal per min)
# should probably have cp's named inlet-cp and outlet-cp for consistency
feed_pump_out_props = builder.add_properties(feed_pump['out'], ['flow-rate', 'pressure'], [18, 14])

# TODO: I wonder if it would be helpful to model a subsystem for valve assemblages we may want to view as a group
# Is status the right representation for state? Not exactly sure what that means
b3w2_valve = builder.create_equipment('b3w2-valve','valve')
b3w2_valve_outlet = builder.add_connection_point(b3w2_valve, 'outlet-cp')
b3w2_valve_prop = builder.add_properties(b3w2_valve['name'], ['status'], [118])
builder.connect_equipment(feed_pump, b3w2_valve)
builder.add_to_connection(b3w2_valve,waste, source_is_conn = False)

# TODO: is b1 arrow going back to b3w2 wrong?
b1_valve = builder.create_equipment('b1-valve','valve')
b1_valve_prop = builder.add_properties(b1_valve['name'], ['status'],[116])
# TODO: Right now modeling this valve in the 'running mode' the valve flow direction flips in backwash mode
builder.connect_equipment(b3w2_valve, b1_valve)

# Connection directions are getting a little confusing, might have to doublecheck this work
v1_valve = builder.create_equipment('v1_valve', 'valve')
v1_valve_inlet = builder.add_connection_point(v1_valve, 'inlet-cp')
builder.connect_equipment(b1_valve, v1_valve)

# TODO: Change medium to compressed air. Maybe add compressed air source
ac_valve = builder.create_equipment('ac_valve', 'valve')
builder.connect_equipment(ac_valve, v1_valve, target_cp=v1_valve_inlet)
builder.add_properties(ac_valve['name'], ['status'],[126])

# v1 to UF
uf = builder.create_equipment('uf-unit','uf-unit')
builder.connect_equipment(v1_valve, uf)
uf_reject_cp = builder.add_connection_point(uf, 'outlet-cp')

reject_valve = builder.create_equipment('reject_valve','valve')
reject_line = builder.connect_equipment(uf, reject_valve)
reject_line_props = builder.add_properties(reject_line['name'], ['pressure'], [16])
reject_valve_air_outlet = builder.add_connection_point(reject_valve, 'outlet-cp')

# TODO: What does valve with an M mean, manual? 
# Not sure if I should also model the air vent valve, is that an actual valve?
# TODO: currently not representing properties for any of the valves (e.g. opening ratio)
# Are some valves automated? for these I would add actuators
# Do we need different moterized and manual versions of things? 
# Should 223P have different motorized and manual versions of things? 
# Should we only have properties for things that are monitored 
air_valve = builder.create_equipment('air_valve', 'valve')
builder.add_properties(air_valve['name'], ['opening-command'], ['Manual Valve Property'])
builder.connect_equipment(reject_valve, air_valve, source_cp=reject_valve_air_outlet)

s2_valve = builder.create_equipment('s2_valve','valve')
builder.add_properties(s2_valve['name'], ['status'],[127])
builder.connect_equipment(reject_valve, s2_valve)

builder.add_to_connection(s2_valve,waste, source_is_conn=False)

uf_out_props = builder.add_properties(uf['out'], ['turbidity'],[34])
uf_out_valve = builder.create_equipment('uf-out-valve','valve')
# Extra unconnected cp? 
builder.add_connection_point(uf_out_valve,'outlet-cp')
# TODO: might want to add human readable comments to more things

b3w1_valve = builder.create_equipment('b3w1_valve', 'valve')
builder.add_properties(b3w1_valve['name'], ['status'],[117])
b3w1_to_bypass_cp = builder.add_connection_point(b3w1_valve,'outlet-cp') 
uf_out_valve_to_b3w1 = builder.connect_equipment(uf_out_valve, b3w1_valve)

sv8_valve = builder.create_equipment('sv8_valve', 'valve')
builder.add_to_connection(uf_out_valve_to_b3w1, sv8_valve)

uf_backwash_pump = builder.create_equipment('uf_backwash_pump', 'pump')
builder.add_properties(uf_backwash_pump['name'], ['status'],[119])
# TODO: check what the two lines in pump to H1 Valve connection mean
h1_valve = builder.create_equipment('h1_valve', 'valve')
builder.connect_equipment(uf_backwash_pump, h1_valve)
# TODO: I have no idea what the Fl (FI?) in the circle after H1 is 
builder.connect_equipment(h1_valve, b3w1_valve)
uf_bypass_valve_2 = builder.create_equipment('uf_bypass_valve', 'valve')
b3w1_to_bypass_conn = builder.connect_equipment(b3w1_valve, uf_bypass_valve_2, source_cp=b3w1_to_bypass_cp)

# Not sure exactl how I want to represent MULT-TOC-1, 46
# Don't know difference between 46 and 74, MULT-TOC-1 and UF-Permeate TOC. Maybe not different since they're labeled on top of each other for same sensor? 
# 46 and 74 are maybe measuring TOC and calculating PPM differently 
uf_to_bypass_props = builder.add_properties(b3w1_to_bypass_conn['name'], ['pressure','flow-rate','ph','toc-ppm', 'toc-ppm'], [15, 19, 48, 46, 74])

uf_bypass_valve_to_waste = builder.create_equipment('uf_bypass_valve_to_waste', 'valve')
builder.connect_equipment(uf_bypass_valve_1, uf_bypass_valve_to_waste)
uf_bypass_valve_to_waste_cp = builder.add_connection_point(uf_bypass_valve_to_waste, 'outlet-cp')
builder.add_to_connection(uf_bypass_valve_to_waste, waste, source_cp=uf_bypass_valve_to_waste_cp, source_is_conn=False)

uf_bypass_valve_2_from_bypass_cp = builder.add_connection_point(uf_bypass_valve_2, 'inlet-cp')
builder.connect_equipment(uf_bypass_valve_to_waste, uf_bypass_valve_2, target_cp=uf_bypass_valve_2_from_bypass_cp)

# TODO: add ability to add comments to connections
ufd_valve = builder.create_equipment('ufd_valve', 'valve')
builder.add_properties(ufd_valve['name'], ['status'],[143])
permeate_to_gac = builder.connect_equipment(uf_bypass_valve_2, ufd_valve)
builder.graph.add((permeate_to_gac['name'], RDFS['comment'], Literal('Permeate to GAC')))

# TODO: should probably change what is returned by add_connection_point. Functions don't all return the same datatype so this will be hard to use consistently
permeate_to_waste_cp = builder.add_connection_point(ufd_valve, 'outlet-cp')
builder.graph.add((ufd_valve[permeate_to_waste_cp], RDFS['comment'], Literal('UF Permeate to Waste')))
builder.add_to_connection(ufd_valve, waste, source_cp=permeate_to_waste_cp, source_is_conn=False)

NameError: name 'builder' is not defined

In [ ]:
builder.graph.serialize("tzahi-model.ttl", format="turtle")

<Graph identifier=86a6d66e-0254-403f-9b14-b1c85f07f1d0 (<class 'rdflib.graph.Graph'>)>